In [13]:
#https://statso.io/food-delivery-time-prediction-case-study/

In [5]:
!wget https://statso.io/wp-content/uploads/2023/01/Delivery-time.zip

--2023-04-10 11:40:00--  https://statso.io/wp-content/uploads/2023/01/Delivery-time.zip
Resolving statso.io (statso.io)... 192.0.78.24, 192.0.78.25
Connecting to statso.io (statso.io)|192.0.78.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 965026 (942K) [application/zip]
Saving to: ‘Delivery-time.zip’

Delivery-time.zip   100%[===================>] 942.41K  2.92MB/s    in 0.3s    

2023-04-10 11:40:00 (2.92 MB/s) - ‘Delivery-time.zip’ saved [965026/965026]



In [8]:
!unzip Delivery-time.zip

Archive:  Delivery-time.zip
   creating: Delivery time/
  inflating: Delivery time/deliverytime.txt  


In [11]:
import pandas as pd
import numpy as np
import plotly.express as px

data = pd.read_csv("/content/Delivery time/deliverytime.txt")


In [12]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  int64  
 3   Delivery_person_Ratings      45593 non-null  float64
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Type_of_order                45593 non-null  object 
 9   Type_of_vehicle              45593 non-null  object 
 10  Time_taken(min)              45593 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 3.8+ MB


In [15]:
data.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Type_of_order                  0
Type_of_vehicle                0
Time_taken(min)                0
dtype: int64

In [16]:
# Calculating distance in KM

In [17]:
# Set the earth's radius (in kilometers)
R = 6371

# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  
# Calculate the distance between each pair of points
data['distance'] = np.nan

for i in range(len(data)):
    data.loc[i, 'distance'] = distcalculate(data.loc[i, 'Restaurant_latitude'], 
                                        data.loc[i, 'Restaurant_longitude'], 
                                        data.loc[i, 'Delivery_location_latitude'], 
                                        data.loc[i, 'Delivery_location_longitude'])

In [18]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min),distance
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24,3.025149
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33,20.183530
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26,1.552758
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21,7.790401
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30,6.210138


In [19]:
figure = px.scatter(data_frame = data, 
                    x="distance",
                    y="Time_taken(min)", 
                    size="Time_taken(min)", 
                    trendline="ols", 
                    title = "Relationship Between Distance and Time Taken")
figure.show()

Output hidden; open in https://colab.research.google.com to view.

### most delivery partners deliver food within 25-30 

In [20]:
figure = px.scatter(data_frame = data, 
                    x="Delivery_person_Age",
                    y="Time_taken(min)", 
                    size="Time_taken(min)", 
                    color = "distance",
                    trendline="ols", 
                    title = "Relationship Between Time Taken and Age")
figure.show()

Output hidden; open in https://colab.research.google.com to view.

### young delivery partners take less time to deliver the food compared to the elder partners.

In [21]:
figure = px.scatter(data_frame = data, 
                    x="Delivery_person_Ratings",
                    y="Time_taken(min)", 
                    size="Time_taken(min)", 
                    color = "distance",
                    trendline="ols", 
                    title = "Relationship Between Time Taken and Ratings")
figure.show()

Output hidden; open in https://colab.research.google.com to view.

### higher ratings take less time to deliver the food compared to partners with low ratings.

In [22]:
fig = px.box(data, 
             x="Type_of_vehicle",
             y="Time_taken(min)", 
             color="Type_of_order")
fig.show()

### there is not much difference between the time taken by delivery partners depending on the vehicle they are driving and the type of food they are delivering

In [23]:
#splitting data
from sklearn.model_selection import train_test_split
x = np.array(data[["Delivery_person_Age", 
                   "Delivery_person_Ratings", 
                   "distance"]])
y = np.array(data[["Time_taken(min)"]])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.10, 
                                                random_state=42)



In [56]:
xtrain.shape

(41033, 3)

In [24]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()
lr.fit(xtrain,ytrain)

LinearRegression()

In [25]:
ytest_pred = lr.predict(xtest)

In [26]:
from sklearn.metrics import r2_score 
r2score = r2_score(ytest,ytest_pred)
print(r2score)

0.1824330375484946


Predicted Delivery Time in Minutes =  25.430727365417972  Actual Time: 13


(41033, 3)

# Validate Against the Test Data

In [60]:
a,b,d,time = data.iloc[115][['Delivery_person_Age', 'Delivery_person_Ratings', 'distance','Time_taken(min)']] 

pred_time = lr.predict([[a,b,c]])[0][0]

print("Predicted Delivery Time in Minutes = ", pred_time," Actual Time:", time)

Predicted Delivery Time in Minutes =  25.430727365417972  Actual Time: 13


# Predict Now

In [61]:
print("Food Delivery Time Prediction")
a = int(input("Age of Delivery Partner: "))
b = float(input("Ratings of Previous Deliveries: "))
c = int(input("Total Distance: "))

features = np.array([[a, b, c]])
print("Predicted Delivery Time in Minutes = ", lr.predict(features))

Food Delivery Time Prediction
Age of Delivery Partner: 4
Ratings of Previous Deliveries: 4
Total Distance: 14
Predicted Delivery Time in Minutes =  [[20.47787432]]
